# Agente Search

 - Para esse primeiro estudo decidi criar um agente simples, que inicialmente faz busca na web para se completar de um conhecimento atual sugerido.

- Especificações:

1.   Agente React
2.   Tools: busca_web
3. LLM utilizada para testes: llama3-70b-8192
4. Nº de resultados a cada busca_web = 3

**Obs:** O intuito é do agente fazer pesquisas na web para responder querys específicas dentro de um contexto. Por exemplo: Quero o meio de contato de algumas lanchonetes específicas do meu bairro, ela é capaz de pesquisar na web e filtrar os meios de contatos que estão a disposição, sendo assim, respondendo o que foi solicitado.


# Dependências

In [1]:
%pip install groq
%pip install google-search-results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 336.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=6516d4d99b62041138128e9fb1a0d2de79eddb9fcb11ac70b597468e01696b31
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [2]:
import os
os.environ["GROQ_API_KEY"] = "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"

# importando a API do GROQ

In [3]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),  #Conectando API do GROQ
)


# Criando o Agente

In [4]:
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system        # Inicializando parâmetros iniciais do agente (client, system (prompt), messages)
    self.messages = []
    if system is not None:
      self.messages.append({"role": "system", "content": self.system}) # Se mensagem não for vazia, add -> messages

  def __call__(self, message=""): # Função call do agente (Será inicializada sempre que o agente for chamado)
    if message:
      self.messages.append({"role": "user", "content": message}) # Add mensagem do user em message
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result}) # Add mensagem do sistema para o agente
    return result

  def execute(self):
    completion = client.chat.completions.create(
    model="llama3-70b-8192", messages=self.messages  # Atribuindo toda a lista de mensagens ("memória") para o Agente
    )
    return completion.choices[0].message.content #Retornando a primeira saída do agente

# System_prompt

In [5]:
system_prompt = """Você corre em um loop de Pensamento, Ação, PAUSA, Observação.
No final do loop, você emite uma Resposta
Use Pensamento para descrever seus pensamentos sobre a pergunta que lhe foi feita.
Use Ação para executar uma das ações disponíveis para você - então retorne PAUSA.
A Observação será o resultado da execução dessas ações.

Você irá atuar como um assistente de pesquisa para ajudar a explicar conceitos com base em informações encontradas na web. Aqui estão as etapas a seguir:

- Primeiro, você receberá um questionamento
- Pesquise com busca_web o que foi pedido
- Se necessário busque novamente para mais informações
- Volte a pergunta e veja se já possui a resposta
- Responda com certeza o que foi pedido

Ação

busca_web - Busca na web sobre o assunto pedido, retornando os resultados encontrados

Exemplo:

Query: O que é Rede Neural e seus principais topicos dentro de IA
Pensamento: Preciso buscar sobre Rede Neural
Ação: busca_web - Assuntos de Rede Neural
PAUSA

Observação - Resultado web

2º iteração

Pensamento: Será se a informação obtida é o necessário? - Não
Ação - busca_web - Redes Neurais profundas 2025
PAUSA

Observação - Resultado web

3º iteração

Pensamento: Será se a informação obtida é o necessário? - Sim -> Produzir resumo do que foi pedido
Resposta:

Resposta:Redes neurais são sistemas de computação com nós interconectados que funcionam como os neurônios do cérebro humano.
Usando algoritmos, elas podem reconhecer padrões escondidos e correlações em dados brutos, agrupá-los e classificá-los, e – com o tempo – aprender e melhorar continuamente.

""".strip()

# AÇÕES

In [12]:
from serpapi import GoogleSearch

def busca_web(msg) -> dict:
    """
    Função que utiliza a SerpApi para pesquisar um conceito no Google e retorna os resultados.

    Parâmetros:
    msg (str): O termo ou conceito a ser pesquisado.

    Returns:
        dict: Um dicionário com os resultados da busca.
    """
    params = {
          "q": msg,
          "api_key": "74bdb667acb866a40a789e1b21da51606874c83f317d9825df922c63991df746",
          "engine": "google",
          "num": 3,  # define o número de resultados desejados
      }
    search = GoogleSearch(params)
    results = search.get_dict()

    if "organic_results" in results:
        conteudo = ""
        for i, item in enumerate(results["organic_results"]):
            title = item.get("title", "Sem título")
            snippet = item.get("snippet", "Sem descrição")       # Processo de formatação do que foi obtido na busca
            conteudo += f"{i+1}. {title}\n{snippet}\n\n"         # Retornando conteudo (Conteúdo filtrado)
        return conteudo.strip()
    else:
        return "Nenhum resultado encontrado."


# Definindo LOOP

In [7]:
import re

def agent_loop(max_iterations, system, query): # Inicializando o loop do agente
  agent = Agent(client, system_prompt)
  tools = ["busca_web"]    # Definindo tools
  next_prompt = query
  i = 0                                       # Contagem do loop

  while i < max_iterations:
    result = agent(next_prompt)
    print(result)
    i += 1

    if "Resposta:" in result:
        break

    if "busca_web" in result:      # Se no resultado do agente contiver o uso da tool "busca_web" ele irá entrar nessa condição
       action = re.findall(r"Ação: busca_web\s*-\s*(.+)", result, re.IGNORECASE)[0]  # Selecionando apenas o nome da TOOL especificada
       next_prompt = f"Observação: {busca_web(action)}"  # Adicionando o resultado da busca no próximo prompt do agente

    else:
        next_prompt = f"Observação: Tool not found" # Caso não exista a tool mencionada ele insere uma mensagem default

    print(next_prompt)
    continue

# Testes

In [13]:
teste = agent_loop(max_iterations = 30, system = system_prompt, query = "Me dê o nome de 10 sanduicherias famosas de goiania e seus respectívos meio de contato (número)")

Pensamento: Preciso buscar sobre sanduicherias famosas em Goiânia para fornecer a lista solicitada.

Ação: busca_web - Sanduicherias famosas em Goiânia
PAUSA
Observação: 1. As 10 Melhores Hamburguerias de Goiânia em 2023
Lifebox · Madero · Matturado · Mafuá Burger · Meatz Burger n' beer · La Brasa · 180G Burger · Studio Burger.

2. Os melhores: Goiânia - Hambúrguer
Os melhores: Goiânia - Hambúrguer · 1. Madero. (1.144). Aberto agora · 2. Lifebox Burger - Setor Oeste. (521). Aberto agora · 3. Outback Steakhouse - Shopping ...
Pensamento: Encontrei algumas sanduicherias famosas em Goiânia, mas preciso mais informações sobre elas e seus respectivos meios de contato. Além disso, observei que a lista não inclui somente sanduicherias, mas também restaurantes que servem hambúrgueres.

Ação: busca_web - Contatos das sanduicherias em Goiânia
PAUSA
Observação: 1. Sanduicheria Smiley🍔 (@sanduicheriasmiley10)
A tradicional sanduicheria de Goiânia, famosa da 10! Sanduíches, cremes e sucos ☎️ (62) 4

In [14]:
teste = agent_loop(max_iterations = 30, system = system_prompt, query = "Me fale sobre os 5 whiskys mais caros da web, falando sobre seu preço, pontos negativos e positivos")

Pensamento: Preciso buscar sobre os whiskys mais caros da web.

Ação: busca_web - Os 5 whiskys mais caros do mundo.

PAUSA
Observação: 1. Quais são os 5 whiskys mais caros do mundo
1. Isabella's Islay – US$ 6,2 milhões. O whisky mais caro do mundo é, na verdade, uma arte de joalheria. Sua garrafa de cristal é coberta por ...

2. What are the most expensive whiskies in the world?
What are the most expensive whiskies in the world? · The Emerald Isle Collection – $2.8m · The Macallan Fine & Rare Collection 1926 60-Year-Old – ...

3. Até R$ 182 mil cada: os 10 whiskies mais premium de 2023
1. The Dalmore Cask Curation Series, The Sherry Edition: US$ 37.000 (R$ 182 mil) · 2. Dewars Double Double 37: US$ 1.800 (R$ 8.800) · 3. Aberfeldy ...
Pensamento: Okay, encontrei algumas listas de whiskys caros, mas preciso de mais informações sobre cada um deles, especialmente sobre seus pontos negativos e positivos.

Ação: busca_web - Informações detalhadas sobre Isabella's Islay, The Emerald Isle Coll